# Twitter Query Expansion
In this demo file, the project **Twitter Query Expansion** is ...

## Download Models
First, different models for text processing and word embeddings are downloaded. 

For text processing SpaCy is utilized. The word embeddings of Word2Vec and Fasttext are 

In [1]:
SPACY_MODEL = "de_core_news_lg"

In [ ]:
import subprocess

# download the respective SpaCy model
subprocess.run(f"python -m spacy download {SPACY_MODEL}", shell=True)

Download FastText model

In [ ]:
# Download german model from fasttext website
subprocess.run("wget -P ./data/fasttext https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.bin.gz", shell=True)

# unzip the fasttext model
subprocess.run(f"gunzip -d ./data/fasttext/cc.de.300.bin.gz", shell=True)

Download Word2Vec model

In [ ]:
# Download german model from devmount website
subprocess.run("wget -P ./data/fasttext https://cloud.devmount.de/d2bc5672c523b086/german.model", shell=True)

## Define Queries
The queries on which to evaluate the query expansion are stated.

In [2]:
QUERIES = [
    "@amthor Ist die große Koalition gescheitert unter Merkel? #Groko#SPD #CDU",
    "#Lauterbach Virus! gelogen #Covid19 #Corona"
]

SPACY_MODEL = "de_core_news_lg"

## Set Pipeline Parameters
Modify the following parameters to use different options for the word embeddings and elastic search query.

In [3]:
EMBEDDING_PARAMS = {
    "type": "word2vec",
    "pos_list": ["NOUN","ADJ","VERB"],
    "entity": False,
    "hashtag": True,
    "user": False,
    "num_nearest_terms": 3
}

ELASTIC_PARAMS = {
    "index": "tweets",
    "retweet": False,
    "hashtag_boost": 0.5,
    "tweet_range": ("2021-01-01", "2023-01-01")
}

## Execute Pipeline
Run the Pipeline - the results are stored in the `/out` directory.

In [4]:
# run pipeline
from scripts import pipeline

res = pipeline.run(
    queries=QUERIES, 
    spacy_model=SPACY_MODEL,
    embedding_params=EMBEDDING_PARAMS,
    elastic_params=ELASTIC_PARAMS)

Processing text using SpaCy...
Loading word2vec model...
Connecting to Elastic Search...
Successfully connected to https://localhost:9200
Retrieving Tweets...
Writing results to out/word2vec/09-01-23_18-46-10
Done!


## Inspect Results
Load results and have a look through the retrieved Tweets. 

In [33]:
for tweets in res:
    for tweet in tweets["tweets"]:
        print(tweet["_source"]["txt"], "\n")

Die erste #GroKo in Deutschland vereinte 1966 noch 86,9% der Wähler:innen hinter sich. Das lässt sich heute nicht mal mit einer Mosambik-Koalition erreichen. Aber wenn man #CDU und #FDP einerseits und #SPD und #Grüne anderseits beobachtet, könnte man das glatt als Lösung sehen. 

Aber…. Ach so, das wollten ja #SPD #CDU und #CSU explizit nicht. Obwohl Grüne, FDP und Linke das beantragt hatten. Die #GroKo nimmt es schlicht billigend in Kauf, dass bis zu 300 Abgeordnete mehr in den Bundestag kommen. Es wäre schlicht verheerend. 

Bis zu 1000 Abgeordnete könnte der neue #Bundestag nach der Wahl bekommen! Die #FDP hatte mit anderen Oppositionsparteien eine effektive #Wahlrechtsreform auf den Weg gebracht. Stattdessen hat sich die #GroKo aus #CDUCSU und #SPD für dieses Reförmchen ohne Wirkung gefeiert. @fdp https://t.co/VZJ1k0dbyW 

Auch wenn der @KSTA lieber #SPD statt #FDP schreibt (🤦🏻‍♂️), es gilt: die „epidemische Notlage von nationaler Tragweite“ besteht nicht mehr, und die gesetzliche 